# Evolver Loop 1 Analysis

Analyzing the baseline and identifying optimization opportunities.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
import json

# Tree polygon vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_polygon(x, y, angle):
    poly = Polygon(zip(TX, TY))
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

# Check N=1 in baseline
df = pd.read_csv('/home/submission/submission.csv')
n1_row = df[df['id'] == '001_0'].iloc[0]
print(f"N=1 baseline: x={n1_row['x']}, y={n1_row['y']}, deg={n1_row['deg']}")

# Parse the values
x = float(n1_row['x'][1:])  # Remove 's' prefix
y = float(n1_row['y'][1:])
angle = float(n1_row['deg'][1:])
print(f"Parsed: x={x}, y={y}, angle={angle}")

N=1 baseline: x=s-48.196086194214246, y=s58.770984615214225, deg=s45.0
Parsed: x=-48.196086194214246, y=58.770984615214225, angle=45.0


In [2]:
# For N=1, the position doesn't matter - only the rotation affects bounding box
# Let's find the optimal rotation for a single tree

def get_bbox_side(angle):
    """Get the bounding box side length for a tree at given angle."""
    tree = create_tree_polygon(0, 0, angle)
    bounds = tree.bounds  # (minx, miny, maxx, maxy)
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    return max(width, height)

# Current N=1 score
current_side = get_bbox_side(45.0)
current_score = current_side ** 2 / 1
print(f"Current N=1: angle=45°, side={current_side:.6f}, score={current_score:.6f}")

# Find optimal angle by exhaustive search
best_angle = 0
best_side = float('inf')
for angle_int in range(0, 36000):  # 0.01° increments
    angle = angle_int / 100
    side = get_bbox_side(angle)
    if side < best_side:
        best_side = side
        best_angle = angle

optimal_score = best_side ** 2 / 1
print(f"\nOptimal N=1: angle={best_angle}°, side={best_side:.6f}, score={optimal_score:.6f}")
print(f"Improvement: {current_score - optimal_score:.6f} points")

Current N=1: angle=45°, side=0.813173, score=0.661250



Optimal N=1: angle=45.0°, side=0.813173, score=0.661250
Improvement: 0.000000 points


In [3]:
# Check the baseline per-N scores
with open('/home/code/experiments/000_baseline/metrics.json') as f:
    metrics = json.load(f)

per_n = metrics['per_n_scores']
print("Top 10 N values by score contribution:")
sorted_n = sorted(per_n.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_n[:10]:
    print(f"  N={n}: {score:.6f}")

print(f"\nTotal score: {sum(per_n.values()):.6f}")
print(f"Target: 68.888293")
print(f"Gap: {sum(per_n.values()) - 68.888293:.6f}")

Top 10 N values by score contribution:
  N=1: 0.661250
  N=2: 0.450779
  N=3: 0.434745
  N=5: 0.416850
  N=4: 0.416545
  N=7: 0.399897
  N=6: 0.399610
  N=9: 0.387415
  N=8: 0.385407
  N=15: 0.376949

Total score: 70.615791
Target: 68.888293
Gap: 1.727498


In [4]:
# Calculate theoretical minimum for N=1
# The tree has height 1.0 (from -0.2 to 0.8) and max width 0.7
# At 0° rotation, bbox is 0.7 x 1.0, so side = 1.0
# At 45° rotation, the diagonal increases
# At 90° rotation, bbox is 1.0 x 0.7, so side = 1.0
# The minimum side occurs at some angle where width ≈ height

print("Checking key angles:")
for angle in [0, 30, 45, 60, 90, 120, 135, 150, 180]:
    side = get_bbox_side(angle)
    print(f"  {angle}°: side={side:.6f}, score={side**2:.6f}")

print(f"\nOptimal angle: {best_angle}°")
print(f"Optimal side: {best_side:.6f}")
print(f"Optimal N=1 score: {best_side**2:.6f}")

Checking key angles:
  0°: side=1.000000, score=1.000000
  30°: side=0.903525, score=0.816358
  45°: side=0.813173, score=0.661250
  60°: side=0.903525, score=0.816358
  90°: side=1.000000, score=1.000000
  120°: side=0.903525, score=0.816358
  135°: side=0.813173, score=0.661250
  150°: side=0.903525, score=0.816358
  180°: side=1.000000, score=1.000000

Optimal angle: 45.0°
Optimal side: 0.813173
Optimal N=1 score: 0.661250


In [5]:
# Let's also check N=2 to N=5 to see if there's room for improvement
print("Analyzing N=2 to N=5:")
for n in range(2, 6):
    baseline_score = float(per_n[str(n)])
    # Theoretical minimum: if trees could be packed perfectly
    # For N trees, minimum area ≈ N * tree_area
    tree_area = Polygon(zip(TX, TY)).area
    min_area = n * tree_area
    min_side = np.sqrt(min_area)
    theoretical_min_score = min_side ** 2 / n
    print(f"  N={n}: baseline={baseline_score:.6f}, theoretical_min≈{theoretical_min_score:.6f}, gap={baseline_score - theoretical_min_score:.6f}")

Analyzing N=2 to N=5:
  N=2: baseline=0.450779, theoretical_min≈0.245625, gap=0.205154
  N=3: baseline=0.434745, theoretical_min≈0.245625, gap=0.189120
  N=4: baseline=0.416545, theoretical_min≈0.245625, gap=0.170920
  N=5: baseline=0.416850, theoretical_min≈0.245625, gap=0.171225


In [6]:
# Summary of optimization potential
print("=" * 50)
print("OPTIMIZATION POTENTIAL SUMMARY")
print("=" * 50)

# N=1 improvement
n1_improvement = current_score - optimal_score
print(f"\nN=1 improvement (exact): {n1_improvement:.6f} points")

# Estimate for N=2-10 (assume 10% improvement possible)
n2_10_total = sum(float(per_n[str(n)]) for n in range(2, 11))
print(f"N=2-10 total: {n2_10_total:.6f} points")
print(f"If 10% improvement: {n2_10_total * 0.1:.6f} points")

# Total potential
total_potential = n1_improvement + n2_10_total * 0.1
print(f"\nEstimated total potential from small N: {total_potential:.6f} points")
print(f"Current gap to target: {sum(per_n.values()) - 68.888293:.6f} points")
print(f"Remaining gap after small N optimization: {sum(per_n.values()) - 68.888293 - total_potential:.6f} points")

OPTIMIZATION POTENTIAL SUMMARY

N=1 improvement (exact): 0.000000 points
N=2-10 total: 3.667878 points
If 10% improvement: 0.366788 points

Estimated total potential from small N: 0.366788 points
Current gap to target: 1.727498 points
Remaining gap after small N optimization: 1.360711 points


# Investigating Overlap Issue in N=40

The submission failed with "Overlapping trees in group 040". Let's investigate.

In [ ]:
# Check N=40 for overlaps
def parse_submission_full(df):
    """Parse submission dataframe into dict of n -> list of (x, y, angle)."""
    solutions = {}
    for _, row in df.iterrows():
        id_parts = row['id'].split('_')
        n = int(id_parts[0])
        x = float(row['x'][1:])  # Remove 's' prefix
        y = float(row['y'][1:])  # Remove 's' prefix
        angle = float(row['deg'][1:])  # Remove 's' prefix
        if n not in solutions:
            solutions[n] = []
        solutions[n].append((x, y, angle))
    return solutions

def check_overlaps(trees):
    """Check for overlapping trees."""
    polys = [create_tree_polygon(*t) for t in trees]
    overlaps = []
    for i in range(len(polys)):
        for j in range(i+1, len(polys)):
            if polys[i].intersects(polys[j]):
                intersection = polys[i].intersection(polys[j])
                if intersection.area > 1e-12:  # Non-trivial overlap
                    overlaps.append((i, j, intersection.area))
    return overlaps

# Load and check
df = pd.read_csv('/home/submission/submission.csv')
solutions = parse_submission_full(df)

print("Checking N=40 for overlaps...")
overlaps_40 = check_overlaps(solutions[40])
if overlaps_40:
    print(f"Found {len(overlaps_40)} overlapping pairs:")
    for i, j, area in overlaps_40[:10]:
        print(f"  Trees {i} and {j}: overlap area = {area:.2e}")
else:
    print("No overlaps detected with standard precision")